In [2]:
import os
import pandas as pd
import numpy as np
import traffic
from traffic.core import Traffic
from traffic.data import opensky
from traffic.data import airports
from traffic.data import aircraft
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm 
import importlib
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

***
## Trajs Preprocessing and Classification (Takeoff & Pushback)
***

In [ ]:
# Select time period (e.g. month) to be downloaded.

airport_str = 'LSZH'

# May 2024
trajs_filenames = ['LSZH_MAY24_1.pkl', 'LSZH_MAY24_2.pkl', 'LSZH_MAY24_3.pkl']
start = ["2024-05-01 00:00","2024-05-11 00:00", "2024-05-21 00:00"]
stop = ["2024-05-10 23:59","2024-05-20 23:59", "2024-05-31 23:59"]

# # June 2024
# trajs_filenames = ['LSZH_JUN24_1.pkl', 'LSZH_JUN24_2.pkl', 'LSZH_JUN24_3.pkl']
# start = ["2024-06-01 00:00","2024-06-11 00:00", "2024-06-21 00:00"]
# stop = ["2024-06-10 23:59","2024-06-20 23:59", "2024-06-30 23:59"]

# # July 2024
# trajs_filenames = ['LSZH_JUL24_1.pkl', 'LSZH_JUL24_2.pkl', 'LSZH_JUL24_3.pkl']
# start = ["2024-07-01 00:00","2024-07-11 00:00", "2024-07-21 00:00"]
# stop = ["2024-07-10 23:59","2024-07-20 23:59", "2024-07-31 23:59"]

# # August 2024
# trajs_filenames = ['LSZH_AUG24_1.pkl', 'LSZH_AUG24_2.pkl', 'LSZH_AUG24_3.pkl']
# start = ["2024-08-01 00:00","2024-08-11 00:00", "2024-08-21 00:00"]
# stop = ["2024-08-10 23:59","2024-08-20 23:59", "2024-08-31 23:59"]

# # September 2024
# trajs_filenames = ['LSZH_SEP24_1.pkl', 'LSZH_SEP24_2.pkl', 'LSZH_SEP24_3.pkl']
# start = ["2024-09-01 00:00","2024-09-11 00:00", "2024-09-21 00:00"]
# stop = ["2024-09-10 23:59","2024-09-20 23:59", "2024-09-30 23:59"]


In [ ]:
import importlib
import agps_funs

# Reload the module to pick up changes
importlib.reload(agps_funs)

from agps_funs import apron_events, takeoff_detection, get_df_movements, add_known_missing_icao24, remove_helos_and_outliers, gs_cumdist_diff, ensure_traffic_datatypes
from agps_config import get_PushStands_LSZH
from traffic.data import aircraft

max_workers=10

for i, filename in enumerate(trajs_filenames): 
    timestamps = pd.date_range(start[i], stop[i], freq="4h")
    data = []
    for t1, t2 in tqdm(zip(timestamps[:-1], timestamps[1:]), total=len(timestamps) - 1, desc="Downloading flights"):
            tmp = opensky.history(start=str(t1), stop=str(t2), bounds=airports[airport_str].shape.convex_hull.buffer(0.1))
            if tmp is not None:
                data.append(tmp.data)
        
    gnd_trajs = pd.concat(data)

    # # Make sure data types of downloaded data are correct
    gnd_trajs=ensure_traffic_datatypes(gnd_trajs)

    # Retain only points near the ground or on the ground
    gnd_trajs = gnd_trajs.query('altitude<4000 or onground')

    # Create Traffic object
    gnd_trajs = Traffic(gnd_trajs)

    # Resample data & add additional columns
    gnd_trajs = gnd_trajs.assign_id().resample('1s').eval(max_workers=max_workers, desc='resampling')
    gnd_trajs = gnd_trajs.aircraft_data()
    gnd_trajs = gnd_trajs.cumulative_distance().eval(max_workers=max_workers, desc='cumdist')

    # Get icaoaircrafttype (if there is any)
    aircraftdata=aircraft.data
    aircraftdata['icao24'] = aircraftdata['icao24'].str.strip().str.lower()
    gnd_trajs = gnd_trajs.merge(aircraftdata[['icao24', 'icaoaircrafttype']], on='icao24', how='left')

    # Classify takeoffs
    gnd_trajs = gnd_trajs.iterate_lazy().pipe(takeoff_detection, 'lszh').eval(max_workers=max_workers, desc='Takeoff Detection')

    # Identify apron events (pushback, taxi)
    standsAreas = get_PushStands_LSZH()
    gnd_trajs = gnd_trajs.iterate_lazy().pipe(apron_events, standsAreas, airport_str, 1).eval(max_workers=max_workers, desc='Apron Events')

    # Determine max-values in compute_gs and cumdist
    gnd_trajs = gnd_trajs.iterate_lazy().pipe(gs_cumdist_diff).eval(max_workers=max_workers, desc='Cumdist-GS-Diff')

    # Save traj data
    gnd_trajs.to_pickle(filename)

    # Get df_movements & save
    df_movements = get_df_movements(gnd_trajs)

    # Add some (of the known missing) icao24
    df_movements = add_known_missing_icao24(df_movements)

    # Remove outliers, helos, etc.
    df_movements = remove_helos_and_outliers(df_movements)

    # Save to pickle
    df_movements.to_pickle(filename.replace('.pkl', '_df_movements.pkl'))
    

***
# Fuel Estimation
***

In [3]:
import agps_funs
from agps_config import AC2CONSIDER
importlib.reload(agps_funs)

# Concatinate df_movements of multiple months
file_paths = ['LSZH_MAY24_1_df_movements.pkl', 'LSZH_MAY24_2_df_movements.pkl', 'LSZH_MAY24_3_df_movements.pkl',
              'LSZH_JUN24_1_df_movements.pkl', 'LSZH_JUN24_2_df_movements.pkl', 'LSZH_JUN24_3_df_movements.pkl',
              'LSZH_JUL24_1_df_movements.pkl', 'LSZH_JUL24_2_df_movements.pkl', 'LSZH_JUL24_3_df_movements.pkl',
              'LSZH_AUG24_1_df_movements.pkl', 'LSZH_AUG24_2_df_movements.pkl', 'LSZH_AUG24_3_df_movements.pkl',
              'LSZH_SEP24_1_df_movements.pkl', 'LSZH_SEP24_2_df_movements.pkl', 'LSZH_SEP24_3_df_movements.pkl',
              ]


df_list = [pd.read_pickle(file) for file in file_paths]
#df_movements = pd.concat(df_list, ignore_index=True).convert_dtypes()
df_movements = pd.concat(df_list, ignore_index=True)

df_movements = df_movements.query('isTakeoff')
df_movements = df_movements[df_movements['typecode'].isin(AC2CONSIDER)]

# Get some aircraft-related data
df_movements = agps_funs.getNengine_df(df_movements)
df_movements = agps_funs.getIdleFF_df(df_movements)
df_movements = agps_funs.getAPUfuelFlow_df(df_movements)

# Estimate Fuel Consumption for Normal Taxi & external AGPS
df_movements = agps_funs.normalTaxiFuel_df(df_movements)
df_movements = agps_funs.extAGPSTaxiFuel_df(df_movements)

df_movements = agps_funs.normalTaxiFuel_df(df_movements, 
                                           startupTime=60,
                                           warmupTime=180,
                                           colNames=['MESengine180', 'MESapu180', 'normTAXIengine180']
                                           )

df_movements.to_pickle('LSZH_MAY_SEP_df_movements.pkl')
df_movements

,flight_id,icao24,callsign,isTakeoff,isPushback,startPushback,startTaxi,lineupTime,taxiDuration,taxiDistance,...,APUhighFF,APUnormalFF,MESengine,MESapu,normTAXIengine,extAGPSapu,extAGPStug,MESengine180,MESapu180,normTAXIengine180
2,AAL93_11938,aae1ea,AAL93,True,False,<NA>,2024-05-05 09:51:52+00:00,2024-05-05 10:04:55+00:00,0 days 00:13:03,1.250523,...,262,238,93.66,16.666667,349.218,51.765,NaN,120.42,20.633333,349.218
3,AAL93_11941,aae597,AAL93,True,False,<NA>,2024-05-01 09:56:35+00:00,2024-05-01 10:03:06+00:00,0 days 00:06:31,1.237633,...,262,238,93.66,16.666667,174.386,25.849444,NaN,120.42,20.633333,174.386
4,AAL93_11945,ab1839,AAL93,True,False,<NA>,2024-05-09 10:06:29+00:00,2024-05-09 10:19:47+00:00,0 days 00:13:18,1.0074,...,262,238,93.66,16.666667,355.908,52.756667,NaN,120.42,20.633333,355.908
5,AAL93_11947,ab1fa7,AAL93,True,False,<NA>,2024-05-03 09:44:34+00:00,2024-05-03 09:56:52+00:00,0 days 00:12:18,0.440692,...,262,238,93.66,16.666667,329.148,48.79,NaN,120.42,20.633333,329.148
6,AAL93_11950,ab236a,AAL93,True,False,<NA>,2024-05-04 09:44:18+00:00,2024-05-04 09:52:56+00:00,0 days 00:08:38,0.69721,...,262,238,93.66,16.666667,231.028,34.245556,NaN,120.42,20.633333,231.028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53354,VLG7516_174,344099,VLG7516,True,False,<NA>,2024-09-27 12:52:21+00:00,2024-09-27 12:55:13+00:00,0 days 00:02:52,0.305183,...,130,110,44.94,8.000000,36.808,5.255556,NaN,57.78,9.833333,36.808
53355,VLG7516_222,344697,VLG7516,True,False,<NA>,2024-09-30 08:53:38+00:00,2024-09-30 08:55:29+00:00,0 days 00:01:51,0.332877,...,130,110,44.94,8.000000,23.754,3.391667,NaN,57.78,9.833333,23.754
53356,VLG7516_247,345645,VLG7516,True,False,<NA>,2024-09-23 08:40:45+00:00,2024-09-23 08:46:04+00:00,0 days 00:05:19,0.881704,...,130,110,44.94,8.000000,68.266,9.747222,NaN,57.78,9.833333,68.266
53357,VLG7516_270,346090,VLG7516,True,False,<NA>,2024-09-25 12:47:38+00:00,2024-09-25 12:52:50+00:00,0 days 00:05:12,0.307059,...,130,110,33.60,8.000000,49.92,9.533333,NaN,43.20,9.833333,49.92
